In [1]:
from torch_geometric.data import InMemoryDataset, Data
import networkx as nx
import torch
import numpy as np
import community as community_louvain
from networkx import karate_club_graph, to_numpy_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import utils
import graph_nets as gn
from tensorflow.keras.utils import to_categorical
from graph_nets import utils_np,utils_tf,modules
import sonnet as snt

In [2]:
class KarateClub(InMemoryDataset):

    def __init__(self, transform=None):
        super(KarateClub, self).__init__('.', transform, None, None)

        G = nx.karate_club_graph()

        x = torch.eye(G.number_of_nodes(), dtype=torch.float)
        order = sorted(list(G.nodes()))
        adjency = to_numpy_matrix(G,nodelist=order)
        
        adj = nx.to_scipy_sparse_matrix(G).tocoo()
        row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
        col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
        edge_index = torch.stack([row, col], dim=0)

        # Compute communities.
        partition = community_louvain.best_partition(G)
        y = torch.tensor([partition[i] for i in range(G.number_of_nodes())])

        # Select a single training node for each community
        # (we just use the first one).
        train_mask = torch.zeros(y.size(0), dtype=torch.bool)
        num_nodes = G.number_of_nodes()
        for i in range(int(y.max()) + 1):
            train_mask[(y == i).nonzero(as_tuple=False)[0]] = True

        data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask,
                    adjency = adjency,num_nodes = num_nodes,
                   number_of_nodes = num_nodes)

        self.data, self.slices = self.collate([data])

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)

In [3]:
dataset= KarateClub()

In [18]:
#adj_sum = np.zeros(34)
#for i in range(34):
    
    
#    adj =   dataset.data.x[i][:] + torch.Tensor(dataset.data.adjency[0][i])
#    adj_sum = np.vstack([adj_sum, adj])

#dan = adj_sum[1:][:]
#dan

In [5]:
def print_graphs_tuple(graphs_tuple):
  print("Shapes of `GraphsTuple`'s fields:")
  #print(graphs_tuple.map(lambda x: x if x is None else x.shape, fields=graphs_tuple.ALL_FIELDS))
  print("\nData contained in `GraphsTuple`'s fields:")
  print("globals:\n{}".format(graphs_tuple.globals))
  print("nodes:\n{}".format(graphs_tuple.nodes))
  print("edges:\n{}".format(graphs_tuple.edges))
  print("senders:\n{}".format(graphs_tuple.senders))
  print("receivers:\n{}".format(graphs_tuple.receivers))
  print("n_node:\n{}".format(graphs_tuple.n_node))
  print("n_edge:\n{}".format(graphs_tuple.n_edge))

In [17]:
graph_nx=dataset[0]

print(graph_nx)
print(f'Number of classes: {len(graph_nx.y.unique())}')
print(f'Number of nodes: {graph_nx.num_nodes}')
print(f'Number of edges: {graph_nx.num_edges}')
print(f'Average node degree: {graph_nx.num_edges / graph_nx.num_nodes:.2f}')
print(f'Number of training nodes: {graph_nx.train_mask.sum()}')
print(f'Training node label rate: {int(graph_nx.train_mask.sum()) / graph_nx.num_nodes:.2f}')
print(f'Contains isolated nodes: {graph_nx.contains_isolated_nodes()}')
print(f'Contains self-loops: {graph_nx.contains_self_loops()}')
print(f'Is undirected: {graph_nx.is_undirected()}')

##Tutaj jest nasz graf

Data(adjency=[[0. 1. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]], edge_index=[2, 156], number_of_nodes=[1], train_mask=[34], x=[34, 34], y=[34])
Number of classes: 4
Number of nodes: 34
Number of edges: 156
Average node degree: 4.59
Number of training nodes: 4
Training node label rate: 0.12
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [20]:


#graphs_tuple = utils_np.data_dicts_to_graphs_tuple([data_dict])
#print_graphs_tuple(graphs_tuple)


In [25]:
num_node = graph_nx.num_nodes
senders = graph_nx.edge_index[0]
senders = tf.make_tensor_proto(senders)
senders = tf.make_ndarray(senders)
receivers = graph_nx.edge_index[1]
receivers = tf.make_tensor_proto(receivers)
receivers = tf.make_ndarray(receivers)
nodes = graph_nx.x
def get_graph_data_dict(num_nodes, num_edges):
    
    return {
    "n_node": num_nodes,
    "nodes": edges,
    "senders": senders,
    "receivers": receivers,
  }
graph_34_nodes_156_edges = get_graph_data_dict(num_nodes=34, num_edges=156)
graphs_tuple = utils_np.data_dicts_to_graphs_tuple([graph_34_nodes_156_edges])
print_graphs_tuple(graphs_tuple)### tutaj tworzymy graph_tuple zeby pozniej gi przekierowac do sieci

Shapes of `GraphsTuple`'s fields:

Data contained in `GraphsTuple`'s fields:
globals:
None
nodes:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
edges:
None
senders:
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1
  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  4  4  4  5  5  5  5
  6  6  6  6  7  7  7  7  8  8  8  8  8  9  9 10 10 10 11 12 12 13 13 13
 13 13 14 14 15 15 16 16 17 17 18 18 19 19 19 20 20 21 21 22 22 23 23 23
 23 23 24 24 24 25 25 25 26 26 27 27 27 27 28 28 28 29 29 29 29 30 30 30
 30 31 31 31 31 31 31 32 32 32 32 32 32 32 32 32 32 32 32 33 33 33 33 33
 33 33 33 33 33 33 33 33 33 33 33 33]
receivers:
[ 1  2  3  4  5  6  7  8 10 11 12 13 17 19 21 31  0  2  3  7 13 17 19 21
 30  0  1  3  7  8  9 13 27 28 32  0  1  2  7 12 13  0  6 10  0  6 10 16
  0  4  5 16  0  1  2  3  0  2 30 32 33  2 33  0  4  5  0  0  3  0  1  2
  3 33 32 33 32 33 

In [29]:
OUTPUT_EDGE_SIZE = 10
OUTPUT_GLOBAL_SIZE = 12
graph_network = modules.RelationNetwork(
    edge_model_fn=lambda: snt.Linear(output_size=OUTPUT_EDGE_SIZE),
    global_model_fn=lambda: snt.Linear(output_size=OUTPUT_GLOBAL_SIZE)
    )
#tutaj definiujemy model, nie do konca rozumiem jakie wartoscie mamy podac
#w OUTPUT_EDGE_SIZE,OUTPUT_GLOBAL_SIZE 

In [28]:

output_graphs = graph_network(graphs_tuple)

print("Output edges size: {}".format(output_graphs.edges.shape[-1]))  # Equal to OUTPUT_EDGE_SIZE
print("Output nodes size: {}".format(output_graphs.nodes.shape[-1]))  # Equal to OUTPUT_NODE_SIZE
print("Output globals size: {}".format(output_graphs.globals.shape[-1]))  # Equal to OUTPUT_GLOBAL_SIZE

AttributeError: 'tuple' object has no attribute 'as_list'